In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

import warnings
warnings.filterwarnings("ignore")

In [2]:
hg_df = pd.read_pickle("hg(1).pkl")
bhg_df = pd.read_pickle("bhg(1).pkl")

In [3]:
bhg_df.head(5)

,标称企业,标称地址,被抽单位,食品,规格,结果,分类,公告日期,任务来源,省份,抽检模式,被抽地址,超标
0,潮州市潮安区庵埠庄陇华利食品厂,潮安区庵埠庄陇大道旁,佛山市南海区大沥弘博食品商行,酸奶慕希饮料,80ml/瓶,甜蜜素(以环己基氨基磺酸计)‖1.32g/kg‖≤0.65g/kg,饮料,2017-05-25,国抽,广东,总局国抽,佛山市南海区大沥大发市场东排601档,甜蜜素
1,东莞市一丰食品厂,广东省东莞市茶山镇卢边村,广州市增城友银商场,下火王果味饮料,600毫升/瓶,菌落总数‖250000CFU/ml‖≤100CFU/ml,饮料,2017-05-25,国抽,广东,总局国抽,广东省广州市增城新塘镇南安市场二楼,菌落总数
2,潮州市潮安区庵埠庄陇华利食品厂,潮安县庵埠庄陇大道旁,佛山市南海区大沥弘博食品商行,土鸡蛋黄,散装,甜蜜素(以环己基氨基磺酸计)‖1.10g/kg‖≤0.65g/kg,糖果制品,2017-05-25,国抽,广东,总局国抽,佛山市南海区大沥大发市场东排601档,甜蜜素
3,潮州市庵埠花泰园食品厂,潮安县庵埠镇厦吴花楼,湛江市赤坎区王富食品商行,桂味杨梅（蜜饯类）,35克/袋,甜蜜素(以环己基氨基磺酸计)‖1.75g/kg‖≤1.0g/kg,水果制品,2017-05-25,国抽,广东,总局国抽,湛江饮料副食品综合批发市场西26幢32号,甜蜜素
4,潮安区正合食品有限公司,潮安区庵埠镇郭四一村新沟片,湛江市赤坎百禄福副食品商行,蜂蜜五味姜（果丹类）,60克/瓶,甜蜜素(以环己基氨基磺酸计)‖7.17g/kg‖≤1.0g/kg,水果制品,2017-05-25,国抽,广东,总局国抽,湛江市饮料副食品综合批发市场东6幢13号1楼,甜蜜素


In [4]:
hg_df.head(1)

,标称企业,标称地址,被抽单位,省份,食品,规格,分类,公告日期,任务来源
0,高桥华盛奶牛场,清远市清城区源潭高桥石古坦（陈兆华）,广州风行牛奶有限公司,广东,生鲜乳（高桥）,散装,乳制品,2016-04-21,省抽


In [5]:
bhg_df.shape, hg_df.shape

((22038, 13), (1006495, 9))

In [6]:
print("正负样本比例：%.0f" % (hg_df.shape[0] / bhg_df.shape[0]))

正负样本比例：46


In [7]:
bhg1_df = bhg_df.drop(["结果", "抽检模式", "被抽地址", "超标"], axis=1)
# bhg1_df["label"] = pd.DataFrame(np.zeros(bhg1_df.shape[0]), dtype=np.int32)
bhg1_df["label"]  = 0
bhg1_df.head(1)

,标称企业,标称地址,被抽单位,食品,规格,分类,公告日期,任务来源,省份,label
0,潮州市潮安区庵埠庄陇华利食品厂,潮安区庵埠庄陇大道旁,佛山市南海区大沥弘博食品商行,酸奶慕希饮料,80ml/瓶,饮料,2017-05-25,国抽,广东,0


In [8]:
hg1_df = hg_df.drop("省份", axis=1)
hg1_df["省份"] = hg_df["省份"]
# hg1_df["label"] = pd.DataFrame(np.ones(hg1_df.shape[0]), dtype=np.int32)
bhg1_df["label"]  = 0
hg1_df.head(3)

,标称企业,标称地址,被抽单位,食品,规格,分类,公告日期,任务来源,省份
0,高桥华盛奶牛场,清远市清城区源潭高桥石古坦（陈兆华）,广州风行牛奶有限公司,生鲜乳（高桥）,散装,乳制品,2016-04-21,省抽,广东
1,广州风行牛奶有限公司,广州市天河区沙太南路342号,广州风行牛奶有限公司,学生饮用奶草莓味牛奶,200ml/盒,乳制品,2016-04-21,省抽,广东
2,广州风行牛奶有限公司,广州市天河区沙太南路342号,广州风行牛奶有限公司,学生饮用奶纯牛奶,200ml/盒,乳制品,2016-04-21,省抽,广东


In [9]:
province_counts = hg1_df["省份"].value_counts()
len( province_counts )

3473

In [10]:
will_drop_index = province_counts[ province_counts < 50 ].index.tolist()
hg1_df = hg1_df.drop( hg1_df.loc[hg1_df["省份"].isin(will_drop_index)].index )
hg1_df.shape

(983951, 9)

In [11]:
hg1_df["省份"].isnull().sum()

0

In [12]:
hg1_df["省份"].value_counts().index

Index(['河南', '山东', '江苏', '广东', '上海', '福建', '四川', '重庆', '北京', '安徽', '湖北', '浙江',
       '黑龙江', '吉林', '贵州', '天津', '湖南', '甘肃', '河北', '江西', '辽宁', '新疆', '陕西',
       '内蒙古', '山西', '云南', '海南', '宁夏', '广西', '兵团', '青海', '厦门', '杭州', '宁波', '绍兴',
       '镇江', '温州', '盐城', '漳州', '嘉兴', '衢州', '台州', '舟山', '湖州', '海宁', '北辰区', '东台',
       '江北区路林综合', '桐乡', '瑞安', '临安', '乐清', '慈溪', '丹阳', '南京', '福州', '苏州',
       '温州菜篮子农副产品批发交易', '常州', '龙海', '金华', '津南区', '浙江丽水', '余姚', '三明', '无锡', '/',
       '长沙', '临海', '龙岩', '泉州', '嵊州', '温岭', '诸暨', '新昌县七星街道江南名茶',
       '钟楼区邹区镇龙潭村委(凌家塘', '泗阳县淮海中路与桃源路西北角', '建德', '句容', '杭州解放路副食品茶叶', '南安',
       '晋江', '常德'],
      dtype='object')

In [13]:
place2province = {
    "温州": "浙江", "镇江": "江苏", "盐城": "江苏", "漳州": "福建", "嘉兴": "浙江", "衢州": "浙江", "台州": "浙江",
    "舟山": "浙江", "湖州": "浙江", "杭州": "浙江", "宁波": "浙江", "绍兴": "浙江", "海宁": "浙江", "北辰区": "天津",
    "津南区": "天津", "东台": "江苏", "江北区路林综合": "浙江", "桐乡": "浙江", "瑞安": "浙江", "临安": "浙江", 
    "乐清": "浙江", "慈溪": "浙江", "南京": "江苏", "丹阳": "江苏", "福州": "福建", "苏州": "江苏", 
    "温州菜篮子农副产品批发交易": "浙江", "常州": "江苏", "龙海": "福建", "金华": "浙江", "浙江丽水": "浙江", 
    "余姚": "浙江", "三明": "福建", "无锡": "江苏", "长沙": "湖南", "临海": "浙江", "龙岩": "福建", "泉州": "福建",
    "嵊州": "浙江", "温岭": "浙江", "诸暨": "浙江", "新昌县七星街道江南名茶": "浙江", "泗阳县淮海中路与桃源路西北角": "江苏",
    "钟楼区邹区镇龙潭村委(凌家塘": "江苏", "建德": "浙江", "句容": "江苏", "南安": "福建", "杭州解放路副食品茶叶": "浙江", 
    "晋江": "福建", "常德": "湖南", "厦门": "福建", '河南': '河南', '山东': '山东', '江苏': '江苏', '广东': '广东',
    '上海': '上海', '福建': '福建', '四川': '四川', '重庆': '重庆', '北京': '北京', '安徽': '安徽', '湖北': '湖北', '浙江': '浙江',
    '黑龙江': '黑龙江', '吉林': '吉林', '贵州': '贵州', '湖南': '湖南', '天津': '天津', '河北': '河北', '甘肃': '甘肃', '江西': '江西', 
    '辽宁': '辽宁', '新疆': '新疆', '陕西': '陕西', '内蒙古': '内蒙古', '山西': '山西', '云南': '云南', '海南': '海南', '宁夏': '宁夏', 
    '广西': '广西', '兵团': '兵团', '青海': '青海', '/': '/'
}

In [14]:
hg2_df = hg1_df

In [15]:
hg2_df["省份"] = hg1_df["省份"].map(place2province)
hg1_df["省份"].value_counts().index

Index(['河南', '山东', '江苏', '广东', '上海', '福建', '四川', '重庆', '北京', '安徽', '浙江', '湖北',
       '黑龙江', '吉林', '贵州', '天津', '湖南', '甘肃', '河北', '江西', '辽宁', '新疆', '陕西',
       '内蒙古', '山西', '云南', '海南', '宁夏', '广西', '兵团', '青海', '/'],
      dtype='object')

In [16]:
hg2_nonan_df = hg2_df.replace("/", np.nan)
hg2_nonan_df.dropna(inplace=True)
hg2_nonan_df.shape, hg2_df.shape

((815283, 9), (983951, 9))

In [17]:
combined_df = pd.concat([hg2_nonan_df, bhg1_df])

In [18]:
# combined_df = combined_df.sample(combined_df.shape[0]).reset_index(drop=True)
combined_df.head()

,label,任务来源,公告日期,分类,标称企业,标称地址,省份,被抽单位,规格,食品
0,NaN,省抽,2016-04-21,乳制品,高桥华盛奶牛场,清远市清城区源潭高桥石古坦（陈兆华）,广东,广州风行牛奶有限公司,散装,生鲜乳（高桥）
1,NaN,省抽,2016-04-21,乳制品,广州风行牛奶有限公司,广州市天河区沙太南路342号,广东,广州风行牛奶有限公司,200ml/盒,学生饮用奶草莓味牛奶
2,NaN,省抽,2016-04-21,乳制品,广州风行牛奶有限公司,广州市天河区沙太南路342号,广东,广州风行牛奶有限公司,200ml/盒,学生饮用奶纯牛奶
3,NaN,省抽,2016-04-21,乳制品,开平市马冈镇粤顺奶牛养殖场,开平市马冈镇黄屋村民委员会陈田新村房屋及山地,广东,广州华农大食品科技有限公司,散装,生鲜乳
4,NaN,省抽,2016-04-21,乳制品,广州华农大食品科技有限公司,广州市天河区华南农业大学内（农场四区21栋）,广东,广州华农大食品科技有限公司,150g/杯,纯白酸牛奶


In [19]:
combined_df["label"].value_counts()

0.0    22038
Name: label, dtype: int64

In [20]:
del hg_df, bhg_df, bhg1_df

In [21]:
name_df = combined_df["标称企业"].value_counts()
name_dic = {}
for key, value in zip(name_df.index, name_df.values):
    name_dic[key] = value
name_dic = name_df.to_dict()

In [22]:
combined_df["标称企业"] = combined_df["标称企业"].map(name_dic)
combined_df.head()

,label,任务来源,公告日期,分类,标称企业,标称地址,省份,被抽单位,规格,食品
0,NaN,省抽,2016-04-21,乳制品,1,清远市清城区源潭高桥石古坦（陈兆华）,广东,广州风行牛奶有限公司,散装,生鲜乳（高桥）
1,NaN,省抽,2016-04-21,乳制品,37,广州市天河区沙太南路342号,广东,广州风行牛奶有限公司,200ml/盒,学生饮用奶草莓味牛奶
2,NaN,省抽,2016-04-21,乳制品,37,广州市天河区沙太南路342号,广东,广州风行牛奶有限公司,200ml/盒,学生饮用奶纯牛奶
3,NaN,省抽,2016-04-21,乳制品,2,开平市马冈镇黄屋村民委员会陈田新村房屋及山地,广东,广州华农大食品科技有限公司,散装,生鲜乳
4,NaN,省抽,2016-04-21,乳制品,9,广州市天河区华南农业大学内（农场四区21栋）,广东,广州华农大食品科技有限公司,150g/杯,纯白酸牛奶


In [23]:
combined_df["标称企业"].value_counts()

combined_df.groupby("标称企业")["label"].mean()

标称企业
1       0.0
2       0.0
3       0.0
4       0.0
5       0.0
6       0.0
7       0.0
8       0.0
9       0.0
10      0.0
11      0.0
12      0.0
13      0.0
14      0.0
15      0.0
16      0.0
17      0.0
18      0.0
19      0.0
20      0.0
21      0.0
22      0.0
23      0.0
24      0.0
25      0.0
26      0.0
27      0.0
28      0.0
29      0.0
30      0.0
       ... 
308     NaN
309     NaN
311     NaN
312     NaN
314     NaN
316     0.0
320     NaN
327     NaN
328     0.0
343     NaN
344     NaN
354     NaN
360     NaN
367     NaN
392     NaN
396     NaN
403     NaN
404     NaN
449     0.0
472     NaN
493     NaN
527     NaN
554     NaN
623     NaN
631     NaN
951     0.0
1171    NaN
2845    0.0
3085    0.0
4366    0.0
Name: label, Length: 260, dtype: float64

In [24]:
combined_df["规格"].value_counts()

散装                    134477
                       33603
25kg/袋                 15084
500ml/瓶                13627
500g/袋                 10614
10kg/袋                 10373
18.9L/桶                 9585
5kg/袋                   8975
200g/袋                  7693
500克/袋                  7595
500mL/瓶                 7322
400g/袋                  7244
250g/袋                  7091
200克/袋                  6881
100克/袋                  6768
100g/袋                  6349
400克/袋                  6253
1kg/袋                   5452
250克/袋                  5236
550ml/瓶                 4845
300g/袋                  4639
150克/袋                  4320
300克/袋                  4191
150g/袋                  4112
500g/瓶                  4092
5L/桶                    3962
2.5kg/袋                 3820
1.8L/瓶                  2825
250mL/盒                 2624
450ml/瓶                 2459
                       ...  
≥480g/盒                    1
330G/罐                     1
240克(4枚）/盒                 1
55g(内装6小包)/袋  

In [25]:
guige2num_dic = {}
guige_count = combined_df["规格"].value_counts()
for key, value in zip(guige_count.index, guige_count.values):
    guige2num_dic[key] = value
    
guige2num_dic = combined_df["规格"].value_counts().to_dict()

In [26]:
combined_df["Spec2num"] = combined_df["规格"].map(guige2num_dic)

In [27]:
combined_df.head()

,label,任务来源,公告日期,分类,标称企业,标称地址,省份,被抽单位,规格,食品,Spec2num
0,NaN,省抽,2016-04-21,乳制品,1,清远市清城区源潭高桥石古坦（陈兆华）,广东,广州风行牛奶有限公司,散装,生鲜乳（高桥）,134477
1,NaN,省抽,2016-04-21,乳制品,37,广州市天河区沙太南路342号,广东,广州风行牛奶有限公司,200ml/盒,学生饮用奶草莓味牛奶,443
2,NaN,省抽,2016-04-21,乳制品,37,广州市天河区沙太南路342号,广东,广州风行牛奶有限公司,200ml/盒,学生饮用奶纯牛奶,443
3,NaN,省抽,2016-04-21,乳制品,2,开平市马冈镇黄屋村民委员会陈田新村房屋及山地,广东,广州华农大食品科技有限公司,散装,生鲜乳,134477
4,NaN,省抽,2016-04-21,乳制品,9,广州市天河区华南农业大学内（农场四区21栋）,广东,广州华农大食品科技有限公司,150g/杯,纯白酸牛奶,53


In [28]:
combined_df["分类"].value_counts()

饮料                           76844
粮食加工品                        69998
糕点                           65433
酒类                           56965
肉制品                          51619
调味品                          44273
食用农产品                        43331
食用油、油脂及其制品                   37518
蔬菜制品                         25947
茶叶及相关制品                      25003
餐饮食品                         24063
乳制品                          20751
水果制品                         19325
速冻食品                         17166
炒货食品及坚果制品                    17011
水产制品                         16790
豆制品                          13768
糖果制品                         13336
粮食及粮食制品                      12551
方便食品                         11841
罐头                            9748
蜂产品                           9081
淀粉及淀粉制品                       8798
肉及肉制品                         6938
饼干                            6603
薯类和膨化食品                       6584
焙烤食品                          6515
蛋制品                           6193
食品添加剂               

In [29]:
# 使用正则表达式删除字符串中的括号及内容
i = 0
classify = []
pattern = re.compile("(\(\S*\))")
pattern1 = re.compile("(\（\S*\）)")
for item in combined_df["分类"]:
    item = pattern.sub("", item)
    item = pattern1.sub("", item)
    classify.append(item)
    i += 1

In [30]:
from collections import Counter
Counter(classify)

Counter({'乳制品': 20751,
         '饮料': 76875,
         '水果及其制品': 2215,
         '罐头': 9748,
         '调味品': 44275,
         '炒货食品及坚果制品': 17017,
         '食用油、油脂及其制品': 37518,
         '餐饮食品': 24063,
         '酒类': 56965,
         '糖果制品': 13443,
         '可可及焙烤咖啡产品': 462,
         '方便食品': 11841,
         '粮食加工品': 69998,
         '蜂产品': 9081,
         '水果制品': 19325,
         '淀粉及淀粉制品': 8798,
         '肉制品': 51625,
         '饼干': 6603,
         '糕点': 65460,
         '食糖': 4772,
         '水产制品': 16790,
         '特殊膳食食品': 3781,
         '速冻食品': 17166,
         '薯类及膨化食品': 3700,
         '茶叶及相关制品': 25003,
         '蔬菜制品': 25947,
         '豆制品': 13768,
         '粮食及粮食制品': 12551,
         '肉及肉制品': 6938,
         '豆类及其制品': 2271,
         '蛋及蛋制品': 646,
         '蔬菜及其制品': 3133,
         '水产品及其制品': 92,
         '焙烤食品': 6609,
         '冷冻饮品': 4336,
         '乳制品及其他含乳食品': 212,
         '茶叶及其相关制品': 4850,
         '糖果及可可制品': 1467,
         '食品添加剂': 5179,
         '食用油，油脂及其制品': 9,
         '薯类和膨化食品': 6584

In [31]:
combined_df["classify"] = classify

In [32]:
temp_classify_count = combined_df["classify"].value_counts()

In [33]:
temp_classify_count.value_counts()

1        88
2        37
3        30
4        19
5        17
6        14
7        12
9        11
10       11
11       10
19       10
18        8
8         8
30        7
12        6
24        6
16        6
13        6
21        6
39        5
26        5
25        5
47        5
17        5
55        5
20        4
59        4
33        4
35        4
15        4
         ..
124       1
122       1
121       1
119       1
1141      1
116       1
115       1
144       1
1467      1
6584      1
192       1
216       1
212       1
51625     1
209       1
2253      1
1010      1
201       1
197       1
191       1
157       1
1205      1
177       1
175       1
25003     1
2215      1
166       1
165       1
164       1
27        1
Name: classify, Length: 254, dtype: int64

In [34]:
temp_index = temp_classify_count[temp_classify_count == 1].index

In [35]:
combined_df.loc[combined_df["分类"].isin(temp_index), "label"].sum()

0.0

In [36]:
classify_dict = temp_classify_count.to_dict()

In [37]:
combined_df["classify"] = combined_df["分类"].map(classify_dict)

In [38]:
combined_df.head()

,label,任务来源,公告日期,分类,标称企业,标称地址,省份,被抽单位,规格,食品,Spec2num,classify
0,NaN,省抽,2016-04-21,乳制品,1,清远市清城区源潭高桥石古坦（陈兆华）,广东,广州风行牛奶有限公司,散装,生鲜乳（高桥）,134477,20751.0
1,NaN,省抽,2016-04-21,乳制品,37,广州市天河区沙太南路342号,广东,广州风行牛奶有限公司,200ml/盒,学生饮用奶草莓味牛奶,443,20751.0
2,NaN,省抽,2016-04-21,乳制品,37,广州市天河区沙太南路342号,广东,广州风行牛奶有限公司,200ml/盒,学生饮用奶纯牛奶,443,20751.0
3,NaN,省抽,2016-04-21,乳制品,2,开平市马冈镇黄屋村民委员会陈田新村房屋及山地,广东,广州华农大食品科技有限公司,散装,生鲜乳,134477,20751.0
4,NaN,省抽,2016-04-21,乳制品,9,广州市天河区华南农业大学内（农场四区21栋）,广东,广州华农大食品科技有限公司,150g/杯,纯白酸牛奶,53,20751.0


In [39]:
province = combined_df["省份"].unique()

In [40]:
province

array(['广东', '河南', '山西', '北京', '湖北', '吉林', '浙江', '江苏', '海南', '新疆', '兵团',
       '陕西', '内蒙古', '福建', '天津', '上海', '宁夏', '山东', '重庆', '贵州', '甘肃', '湖南',
       '四川', '黑龙江', '安徽', '辽宁', '河北', '广西', '青海', '云南', '江西'],
      dtype=object)

In [41]:
province_dict = {}
for value, key in enumerate(province):
    province_dict[key] = value+1

In [42]:
combined_df["Province"] = combined_df["省份"].map(province_dict)

In [43]:
combined_df.head()

,label,任务来源,公告日期,分类,标称企业,标称地址,省份,被抽单位,规格,食品,Spec2num,classify,Province
0,NaN,省抽,2016-04-21,乳制品,1,清远市清城区源潭高桥石古坦（陈兆华）,广东,广州风行牛奶有限公司,散装,生鲜乳（高桥）,134477,20751.0,1
1,NaN,省抽,2016-04-21,乳制品,37,广州市天河区沙太南路342号,广东,广州风行牛奶有限公司,200ml/盒,学生饮用奶草莓味牛奶,443,20751.0,1
2,NaN,省抽,2016-04-21,乳制品,37,广州市天河区沙太南路342号,广东,广州风行牛奶有限公司,200ml/盒,学生饮用奶纯牛奶,443,20751.0,1
3,NaN,省抽,2016-04-21,乳制品,2,开平市马冈镇黄屋村民委员会陈田新村房屋及山地,广东,广州华农大食品科技有限公司,散装,生鲜乳,134477,20751.0,1
4,NaN,省抽,2016-04-21,乳制品,9,广州市天河区华南农业大学内（农场四区21栋）,广东,广州华农大食品科技有限公司,150g/杯,纯白酸牛奶,53,20751.0,1


In [44]:
i = 0
spec_list = []
pattern = re.compile("千克")
pattern1 = re.compile("克")
pattern2 = re.compile("L")
pattern3 = re.compile("毫升")
pattern4 = re.compile("升")
pattern5 = re.compile("公斤")
pattern6 = re.compile("([0-9]+\w+/\w)")
pattern7 = re.compile("\（.*?\）")
pattern8 = re.compile("\(.*?\)")
pattern9 = re.compile("\（.*?\)")
pattern10 = re.compile("\(.*?\）")
pattern11 = re.compile("\s")

for item in combined_df["规格"]:
    item = pattern.sub("kg", item)
    item = pattern1.sub("g", item)
    item = pattern2.sub("l", item)
    item = pattern3.sub("ml", item)
    item = pattern4.sub("l", item)
    item = pattern5.sub("kg", item)
    temp_obj = pattern6.search(item)
    if temp_obj:
        item = temp_obj.group(1)
    item = pattern7.sub("", item)
    item = pattern8.sub("", item)
    item = pattern9.sub("", item)
    item = pattern10.sub("", item)
    item = pattern11.sub("", item)
    spec_list.append(item)
    i += 1

In [45]:
combined_df["Spec"] = spec_list

In [46]:
combined_df.head()

,label,任务来源,公告日期,分类,标称企业,标称地址,省份,被抽单位,规格,食品,Spec2num,classify,Province,Spec
0,NaN,省抽,2016-04-21,乳制品,1,清远市清城区源潭高桥石古坦（陈兆华）,广东,广州风行牛奶有限公司,散装,生鲜乳（高桥）,134477,20751.0,1,散装
1,NaN,省抽,2016-04-21,乳制品,37,广州市天河区沙太南路342号,广东,广州风行牛奶有限公司,200ml/盒,学生饮用奶草莓味牛奶,443,20751.0,1,200ml/盒
2,NaN,省抽,2016-04-21,乳制品,37,广州市天河区沙太南路342号,广东,广州风行牛奶有限公司,200ml/盒,学生饮用奶纯牛奶,443,20751.0,1,200ml/盒
3,NaN,省抽,2016-04-21,乳制品,2,开平市马冈镇黄屋村民委员会陈田新村房屋及山地,广东,广州华农大食品科技有限公司,散装,生鲜乳,134477,20751.0,1,散装
4,NaN,省抽,2016-04-21,乳制品,9,广州市天河区华南农业大学内（农场四区21栋）,广东,广州华农大食品科技有限公司,150g/杯,纯白酸牛奶,53,20751.0,1,150g/杯


In [47]:
combined_df.head()

,label,任务来源,公告日期,分类,标称企业,标称地址,省份,被抽单位,规格,食品,Spec2num,classify,Province,Spec
0,NaN,省抽,2016-04-21,乳制品,1,清远市清城区源潭高桥石古坦（陈兆华）,广东,广州风行牛奶有限公司,散装,生鲜乳（高桥）,134477,20751.0,1,散装
1,NaN,省抽,2016-04-21,乳制品,37,广州市天河区沙太南路342号,广东,广州风行牛奶有限公司,200ml/盒,学生饮用奶草莓味牛奶,443,20751.0,1,200ml/盒
2,NaN,省抽,2016-04-21,乳制品,37,广州市天河区沙太南路342号,广东,广州风行牛奶有限公司,200ml/盒,学生饮用奶纯牛奶,443,20751.0,1,200ml/盒
3,NaN,省抽,2016-04-21,乳制品,2,开平市马冈镇黄屋村民委员会陈田新村房屋及山地,广东,广州华农大食品科技有限公司,散装,生鲜乳,134477,20751.0,1,散装
4,NaN,省抽,2016-04-21,乳制品,9,广州市天河区华南农业大学内（农场四区21栋）,广东,广州华农大食品科技有限公司,150g/杯,纯白酸牛奶,53,20751.0,1,150g/杯


In [48]:
temp = combined_df["Spec"].value_counts()

In [49]:
spec_dic = {}
for key, value in zip(temp.index, temp.values):
    spec_dic[key] = value

In [50]:
combined_df.head()

,label,任务来源,公告日期,分类,标称企业,标称地址,省份,被抽单位,规格,食品,Spec2num,classify,Province,Spec
0,NaN,省抽,2016-04-21,乳制品,1,清远市清城区源潭高桥石古坦（陈兆华）,广东,广州风行牛奶有限公司,散装,生鲜乳（高桥）,134477,20751.0,1,散装
1,NaN,省抽,2016-04-21,乳制品,37,广州市天河区沙太南路342号,广东,广州风行牛奶有限公司,200ml/盒,学生饮用奶草莓味牛奶,443,20751.0,1,200ml/盒
2,NaN,省抽,2016-04-21,乳制品,37,广州市天河区沙太南路342号,广东,广州风行牛奶有限公司,200ml/盒,学生饮用奶纯牛奶,443,20751.0,1,200ml/盒
3,NaN,省抽,2016-04-21,乳制品,2,开平市马冈镇黄屋村民委员会陈田新村房屋及山地,广东,广州华农大食品科技有限公司,散装,生鲜乳,134477,20751.0,1,散装
4,NaN,省抽,2016-04-21,乳制品,9,广州市天河区华南农业大学内（农场四区21栋）,广东,广州华农大食品科技有限公司,150g/杯,纯白酸牛奶,53,20751.0,1,150g/杯


In [51]:
combined_df["Spec"] = combined_df["Spec"].map(spec_dic)

In [52]:
combined_df.drop(["规格", "分类", "省份", "Spec2num"], axis=1, inplace=True)
combined_df.head()

,label,任务来源,公告日期,标称企业,标称地址,被抽单位,食品,classify,Province,Spec
0,NaN,省抽,2016-04-21,1,清远市清城区源潭高桥石古坦（陈兆华）,广州风行牛奶有限公司,生鲜乳（高桥）,20751.0,1,134477
1,NaN,省抽,2016-04-21,37,广州市天河区沙太南路342号,广州风行牛奶有限公司,学生饮用奶草莓味牛奶,20751.0,1,1045
2,NaN,省抽,2016-04-21,37,广州市天河区沙太南路342号,广州风行牛奶有限公司,学生饮用奶纯牛奶,20751.0,1,1045
3,NaN,省抽,2016-04-21,2,开平市马冈镇黄屋村民委员会陈田新村房屋及山地,广州华农大食品科技有限公司,生鲜乳,20751.0,1,134477
4,NaN,省抽,2016-04-21,9,广州市天河区华南农业大学内（农场四区21栋）,广州华农大食品科技有限公司,纯白酸牛奶,20751.0,1,97


In [53]:
source_counts = combined_df["任务来源"].value_counts()

source_dict = {}
for i, source in enumerate(source_counts.index):
    source_dict[source] = i + 1
    
combined_df["Source"] = combined_df["任务来源"].map(source_dict)

In [54]:
combined_df.head()

,label,任务来源,公告日期,标称企业,标称地址,被抽单位,食品,classify,Province,Spec,Source
0,NaN,省抽,2016-04-21,1,清远市清城区源潭高桥石古坦（陈兆华）,广州风行牛奶有限公司,生鲜乳（高桥）,20751.0,1,134477,2
1,NaN,省抽,2016-04-21,37,广州市天河区沙太南路342号,广州风行牛奶有限公司,学生饮用奶草莓味牛奶,20751.0,1,1045,2
2,NaN,省抽,2016-04-21,37,广州市天河区沙太南路342号,广州风行牛奶有限公司,学生饮用奶纯牛奶,20751.0,1,1045,2
3,NaN,省抽,2016-04-21,2,开平市马冈镇黄屋村民委员会陈田新村房屋及山地,广州华农大食品科技有限公司,生鲜乳,20751.0,1,134477,2
4,NaN,省抽,2016-04-21,9,广州市天河区华南农业大学内（农场四区21栋）,广州华农大食品科技有限公司,纯白酸牛奶,20751.0,1,97,2


In [55]:
drop_index = combined_df[combined_df["公告日期"].isnull()]["label"].index
combined_df.drop(drop_index, inplace=True)
combined_df.reset_index(drop=True, inplace=True)

In [56]:
def date2season(col):
    # 根据月份得出季节
    new_col = pd.to_datetime(col)
    new_list = []
    for item in new_col:
        month = int(item.month)
        if month in [1, 2, 3]:
            new_list.append(1)
        elif month in [4, 5, 6]:
            new_list.append(2)
        elif month in [7, 8, 9]:
            new_list.append(3)
        else:
            new_list.append(4)
    return new_list

In [57]:
new_list = date2season(combined_df["公告日期"])
combined_df["Season"] = new_list
combined_df.head()

,label,任务来源,公告日期,标称企业,标称地址,被抽单位,食品,classify,Province,Spec,Source,Season
0,NaN,省抽,2016-04-21,1,清远市清城区源潭高桥石古坦（陈兆华）,广州风行牛奶有限公司,生鲜乳（高桥）,20751.0,1,134477,2,2
1,NaN,省抽,2016-04-21,37,广州市天河区沙太南路342号,广州风行牛奶有限公司,学生饮用奶草莓味牛奶,20751.0,1,1045,2,2
2,NaN,省抽,2016-04-21,37,广州市天河区沙太南路342号,广州风行牛奶有限公司,学生饮用奶纯牛奶,20751.0,1,1045,2,2
3,NaN,省抽,2016-04-21,2,开平市马冈镇黄屋村民委员会陈田新村房屋及山地,广州华农大食品科技有限公司,生鲜乳,20751.0,1,134477,2,2
4,NaN,省抽,2016-04-21,9,广州市天河区华南农业大学内（农场四区21栋）,广州华农大食品科技有限公司,纯白酸牛奶,20751.0,1,97,2,2


In [58]:
combined_df["Company"] = combined_df["标称企业"]

In [59]:
combined_df.drop(["标称企业", "食品", "公告日期", "任务来源"], axis=1, inplace=True)

In [60]:
combined_df.columns

Index(['label', '标称地址', '被抽单位', 'classify', 'Province', 'Spec', 'Source',
       'Season', 'Company'],
      dtype='object')

In [61]:
combined_df.isnull().sum()

label       815280
标称地址             0
被抽单位             0
classify      4414
Province         0
Spec             0
Source           0
Season           0
Company          0
dtype: int64

In [62]:
combined_df.head(2)

,label,标称地址,被抽单位,classify,Province,Spec,Source,Season,Company
0,NaN,清远市清城区源潭高桥石古坦（陈兆华）,广州风行牛奶有限公司,20751.0,1,134477,2,2,1
1,NaN,广州市天河区沙太南路342号,广州风行牛奶有限公司,20751.0,1,1045,2,2,37


In [63]:
combined_df['label'] = combined_df['label'].fillna(1.0)

In [64]:
combined_df['Province'].value_counts()

2     73538
18    71840
8     57933
1     55612
16    53056
14    44056
23    39966
4     38362
25    35140
5     30387
24    27531
7     27463
6     24989
20    24419
19    23595
27    23380
15    23146
21    20848
22    20683
31    18434
26    18228
10    13703
12    13519
30    11230
13    10581
3      9558
28     6820
17     6762
9      4471
29     4377
11     3690
Name: Province, dtype: int64

In [65]:
tmp = combined_df['省份'].value_counts()

KeyError: '省份'

In [ ]:
qwe = {}
for key, value in zip(tmp.values, tmp.index):
    qwe[key] = value

In [84]:
tmp1 = combined_df['分类'].value_counts()

KeyError: '分类'

In [ ]:
qqqw = {}
for key, value in zip(tmp1.values, tmp1.index):
    qqqw[key] = value

In [85]:
import pickle
with open('importent_dict.pkl','rb')as f:
    a = pickle.load(f)

In [88]:
a[2]

{'散装': 135937,
 '': 33910,
 '500ml/瓶': 31942,
 '500g/袋': 19118,
 '5kg/袋': 18134,
 '25kg/袋': 16905,
 '200g/袋': 15767,
 '400g/袋': 14667,
 '100g/袋': 13653,
 '250g/袋': 12729,
 '10kg/袋': 11419,
 '9l/桶': 11145,
 '300g/袋': 9467,
 '5l/桶': 8980,
 '150g/袋': 8776,
 '1kg/袋': 8312,
 '550ml/瓶': 7859,
 '5l/瓶': 6769,
 '500g/瓶': 6394,
 '750ml/瓶': 6392,
 '250ml/盒': 6013,
 '450ml/瓶': 5443,
 '8l/瓶': 5170,
 '180g/袋': 5013,
 '80g/袋': 4859,
 '120g/袋': 4402,
 '350g/袋': 4222,
 '200g/盒': 3637,
 '450g/袋': 3602,
 '50g/袋': 3591,
 '90g/袋': 3517,
 '350ml/瓶': 3398,
 '160g/袋': 3215,
 '60g/袋': 3041,
 '480ml/瓶': 2906,
 '400g/盒': 2891,
 '250g/盒': 2700,
 '8l/桶': 2684,
 '70g/袋': 2641,
 '500ml/盒': 2586,
 '15kg/袋': 2491,
 '1l/瓶': 2445,
 '600ml/瓶': 2346,
 '800ml/瓶': 2255,
 '900ml/瓶': 2219,
 '500g/包': 2189,
 '300g/盒': 2176,
 '280g/袋': 2144,
 '18l/桶': 2131,
 '260g/袋': 2054,
 '17l/桶': 2012,
 '330ml/瓶': 2010,
 '600g/袋': 1967,
 '110g/袋': 1934,
 '30g/袋': 1907,
 '200g/包': 1898,
 '400ml/瓶': 1880,
 '320g/袋': 1875,
 '240g/袋': 1857,
 '5

In [66]:
import pickle
with open('importent_dict.pkl','rb')as f:
    qq = pickle.load(f)

In [67]:
qq

[{'饮料': 77945,
  '粮食加工品': 70809,
  '糕点': 66294,
  '酒类': 57666,
  '肉制品': 52429,
  '调味品': 44757,
  '食用农产品': 43592,
  '食用油、油脂及其制品': 37883,
  '蔬菜制品': 26194,
  '茶叶及相关制品': 25214,
  '餐饮食品': 24522,
  '乳制品': 21079,
  '水果制品': 19556,
  '速冻食品': 17513,
  '炒货食品及坚果制品': 17253,
  '水产制品': 16932,
  '豆制品': 13853,
  '糖果制品': 13575,
  '粮食及粮食制品': 12831,
  '方便食品': 12035,
  '罐头': 9866,
  '蜂产品': 9176,
  '淀粉及淀粉制品': 8884,
  '肉及肉制品': 7105,
  '焙烤食品': 6731,
  '饼干': 6659,
  '薯类和膨化食品': 6625,
  '蛋制品': 6247,
  '食品添加剂': 5272,
  '茶叶及其相关制品': 4870,
  '食糖': 4839,
  '冷冻饮品': 4370,
  '特殊膳食食品': 3913,
  '薯类及膨化食品': 3757,
  '蔬菜及其制品': 3200,
  '熟肉制品': 2688,
  '豆类及其制品': 2316,
  '水果及其制品': 2270,
  '畜禽肉及副产品': 2266,
  '食用植物油': 2203,
  '蔬菜': 1842,
  '婴幼儿配方食品': 1727,
  '糖果': 1726,
  '酱腌菜': 1723,
  '大米': 1696,
  '水产品': 1614,
  '糖果及可可制品': 1526,
  '乳基较大婴儿和幼儿配方食品、豆基较大婴儿和幼儿配方食品': 1295,
  '粽子': 1244,
  '发酵面制品': 1170,
  '水产品及水产制品': 1165,
  '乳基婴儿配方食品、豆基婴儿配方食品': 1096,
  '食用油': 1087,
  '保健食品': 1030,
  '畜肉': 989,
  '酱卤肉制品': 959,
  '其他粮食加工品': 880,
  '其他